# Final Projcet - HW5

In [1]:
!nvidia-smi

Sun Nov 16 21:34:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0             52W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
# Environment Preparation
!pip install torchsummaryX==1.1.0
!pip install ruamel.yaml
!pip install torchmetrics[image]
!pip install wandb --quiet
!pip install pytorch_lightning
!pip install torchinfo --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 59.1 MB/s eta 0:00:00


In [3]:
%matplotlib inline

import torch

import os

import time
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import torchsummaryX
import torch.nn as nn
import torch.nn.functional as F
import math
import gc
import glob
import wandb
import yaml
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import get_linear_schedule_with_warmup

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: ", DEVICE)


Device:  cuda


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# --- make sure your hw5_student_starter_code folder is in the Python path ---
import sys
path = "/content/gdrive/MyDrive/11685 Deep Learning/11685 Diffusion Probabilistic Models/hw5_student_starter_code"
if path not in sys.path:
    sys.path.append(path)
os.chdir(path)
print("✅ Current working dir:", os.getcwd())
!ls -la  # optional, shows files


✅ Current working dir: /content/gdrive/MyDrive/11685 Deep Learning/11685 Diffusion Probabilistic Models/hw5_student_starter_code
total 95
drwx------ 2 root root  4096 Oct 31 19:53 configs
drwx------ 2 root root  4096 Oct 30 02:36 experiments
drwx------ 2 root root  4096 Oct 30 01:40 .idea
drwx------ 2 root root  4096 Oct 31 21:12 inference_epoch_9
drwx------ 2 root root  4096 Nov  1 03:32 inference_out
-rw------- 1 root root  8057 Nov  1 03:26 inference.py
drwx------ 2 root root  4096 Oct 31 20:57 inference_tiny
-rw------- 1 root root     0 Oct  6  2024 __init__.py
-rw------- 1 root root 11357 Oct  6  2024 LICENSE
drwx------ 2 root root  4096 Oct 30 01:40 models
drwx------ 2 root root  4096 Oct 30 01:40 pipelines
drwx------ 2 root root  4096 Nov 16 19:39 pretrained
drwx------ 2 root root  4096 Oct 31 19:42 __pycache__
-rw------- 1 root root  1694 Oct  6  2024 README.md
-rw------- 1 root root    38 Oct  6  2024 requirements.txt
drwx------ 2 root root  4096 Oct 31 19:33 schedulers
-rw---

In [6]:
wandb.login(key="bb92443414fd1681061e7493628c58874493a53e", relogin=True)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sarahw2 (sarahw2-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
!mkdir -p /content/data
!tar -xvzf /content/gdrive/MyDrive/11685\ Deep\ Learning/11685\ Diffusion\ Probabilistic\ Models/data/imagenet100_128x128.tar.gz -C /content/data

In [11]:
!python train.py \
  --config configs/ddpm.yaml

11/16/2025 21:43:28 - INFO - __main__ - Training with a single process on 1 device (cuda).
11/16/2025 21:43:28 - INFO - __main__ - Creating dataset
11/16/2025 21:43:28 - INFO - __main__ - Creating model
11/16/2025 21:43:28 - INFO - __main__ - Number of parameters: 4.24M
wandb: Currently logged in as: sarahw2 (sarahw2-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ setting up run rngkdiiy (0.2s)
wandb: ⣽ setting up run rngkdiiy (0.2s)
wandb: ⣾ setting up run rngkdiiy (0.2s)
wandb: Tracking run with wandb version 0.22.3
wandb: Run data is saved locally in /content/gdrive/MyDrive/11685 Deep Learning/11685 Diffusion Probabilistic Models/hw5_student_starter_code/wandb/run-20251116_214329-rngkdiiy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exp-15-ddpm
wandb: ⭐️ View project at https://wandb.ai/sarahw2-carnegie-mellon-university/ddpm
wandb: 🚀 View run at https://wandb.ai/sar

In [ ]:
!python inference.py \
  --config experiments/exp-7-ddim/config.yaml \
  --ckpt experiments/exp-7-ddim/checkpoints/checkpoint_epoch_10.pth > experiments/exp-7-ddim/fid.txt 2>&1

In [ ]:
!python inference.py \
  --config experiments/exp-7-ddim/config.yaml \
  --ckpt experiments/exp-7-ddim/checkpoints/checkpoint_epoch_10.pth \
  --num_images 256 \
  --eval_is \
  --outdir inference_out > experiments/exp-7-ddim/fid.txt 2>&1